### PyTorch Practice3 : CNN for real defect Image
##### (1) PIL(Pillow) 是 Python 中著名影像處理套件: 可用來轉檔、調色、濾鏡、浮水印甚至創造圖片一堆的功能

In [ ]:
from PIL import Image
# 1. 轉檔 : Image.open() 讀取圖片檔  =================================================================
#           save("FILE NAME","FORMAT") 用來寫入圖片檔，format 其實非必須，PIL會自動透過常見副檔名來判斷格式。
#           常用的 PIL 支援格式 :  BMP ,EPS , GIF , JPEG, PNG ,TIFF , PDF
im = Image.open('./data_manufacture/train/Cr/cr_51.bmp') ;im.save("test.png","png")
im = Image.open('./data_manufacture/train/Cr/cr_51.bmp') ;im.save("test.jpg",'jpeg')
# 2. 修改突變尺寸 :im.resize ==============================================
im = Image.open('./data_manufacture/train/Cr/cr_51.bmp') ; print (im.size); width = 400
ratio = float(width)/im.size[0]
height = int(im.size[1]*ratio)
nim = im.resize( (width, height), Image.Resampling.BILINEAR ); nim.show() ; print (nim.size)
nim.save("test_resized.jpg")
# 3.JPG 壓縮 :save()函式中加入 quality=80, subsampling=0這兩個參數即可壓縮檔案，通常 quality 會設定80或90。
img = Image.open('./data_manufacture/train/Cr/cr_51.bmp'); img.save("test_c.jpg",quality=80,subsampling=0)
# 4. 圖片旋轉 transpose() : Image.FLIP_LEFT_RIGHT (左右翻轉) , Image.FLIP_TOP_DOWN (上下翻轉)
#                  Image.ROTATE_90 (旋轉90度) , Image.ROTATE_180 (旋轉180度) ,Image.ROTATE_270 (旋轉270度)
img = Image.open('./data_manufacture/train/Cr/cr_51.bmp')
imgR =img.transpose(Image.Transpose.ROTATE_180) ; imgR.save("test_R90.jpg")
# 5.製作縮圖 : PIL有一個專門製作縮圖的方法thumbnail()
img = Image.open( './data_manufacture/train/Cr/cr_51.bmp' )
img.thumbnail( (100,100) ) #指定長與寬並進行縮圖製作
img.save( "test_thumbnail.jpg" ) ; print (img.size)
# 6. 加入濾鏡 : PIL 本身就有提供一些濾鏡可以使用,img.fliter() 即可加入濾鏡
# img.filter(ImageFilter.FIND_EDGES)
from PIL import ImageFilter
img = Image.open('./data_manufacture/train/Cr/cr_51.bmp' )
img_f = img.filter(ImageFilter.FIND_EDGES)
img_f.save("test_rf.jpg"); img_f.show()

### torchvision.transforms()
* Transforms are common image transformations.
* They can be chained together using Compose.
* Functional transforms give fine-grained control over the transformations. <br>
<b> transforms.Compose([   <br>
    transforms.CenterCrop(10),    <br>
    transforms.Normalize((0.5, 0.4, 0.5), (0.5, 0.5, 0.5)),    
    transforms.RandomHorizontalFlip(p=0.5),    <br>
    transforms.RandomRotation(),    <br>
    transforms.RandomApply(random_transforms, p=0.3),   <br>
    transforms.ToTensor() ])</b>  
1. class torchvision.transforms.Scale(size, interpolation=2)
2. class torchvision.transforms.RandomCrop(size, padding=0):Crop the given PIL image at a random location.
3. class torchvision.transforms.RandomHorizontalFlip(p=0.5):  <br>
    Horizontally flip the given image randomly with a given probability.
4. class torchvision.transforms.RandomRotation
5. class torchvision.transforms.Resize(size):<br>
   Resize the input PIL image to the given size, usually used with training_testing set
6. class torchvision.transforms.RandomApply
7. class torchvision.transforms.ToTensor
8. class torchvision.transforms.Normalize(mean, std):Only use Normalize with Tensor, not PIL.image
9. class torchvision.transforms.ToTensor // HWC > CHW : Convert PIL.image to Tensor

In [ ]:
# torchvision模組import :
#1.  Numpy image 和 PIL image轉換
from torchvision import transforms
from torchvision.transforms import functional as TF
import numpy as np
from PIL import Image
img_pil = Image.open('./test1.png')
img_pil.show()
print(img_pil.size)
trans_toPIL = transforms.ToPILImage() # 將  "pytoch tensor" 或是  "numpy.ndarray" 轉換成 PIL Image.
img_np = np.asarray(img_pil) # 將PIL image轉換成  "numpy.ndarray" 
print('image type before convert:{}'.format(type(img_np)))
print('img_np.shape =',img_np.shape) ; img_pil = trans_toPIL(img_np)
print('image type after convert:{}'.format(type(img_pil)));img_pil

In [ ]:
# 2. 影像 Normalize
mean =[0.5, 0.5, 0.5] # color image : [0.5, 0.5, 0.5]
std = [0.1, 0.1, 0.1]  # color image :[0.1, 0.1, 0.1]
transform = transforms.Compose([     # transforms.Compose : 將 幾個處理寫在裡面
    transforms.ToTensor(),transforms.Normalize(mean, std), transforms.ToPILImage() ])
img_pil_normal = transform(img_pil)
img_pil_normal

In [ ]:
#  3.影像 Resize : size: 可以設定一個固定長寬值，也可以長寬分別設定 ex: size=200 或是 size = (height, width) = (50,40)
# interpolation: 圖在縮放採用的插值方法，default為PIL.Image.BILINEAR 
# 還有其他方法PIL.Image.NEAREST, PIL.Image.BILINEAR and PIL.Image.BICUBIC.可以選擇
size = 200
transform =  transforms.Resize(size)
new_img = transform(img_pil)
print(new_img.size)
new_img.show()
size = (200, 250)
transform =  transforms.Resize(size)
new_img = transform(img_pil)
new_img

In [ ]:
# ４．影像 CenterCrop : 以圖片(PIL Image)中心點往外延伸設定的大小(size)範圍進行圖像切割。
#  ex: size=200 則是以中心點出來，長寬個擷取200個pixels。
# size = (height, width) = (200,300)，長擷取200個pixel，寬擷取300個pixels
size = 200
transform = transforms.Compose([
    transforms.CenterCrop(size),])
new_img = transform(img_pil)
new_img.show()
size = (200,100)
transform = transforms.Compose([
    transforms.CenterCrop(size),])
new_img = transform(img_pil)
new_img

In [ ]:
#　5. 影像 RandomCrop　：以圖片(PIL Image)中隨機裁減一塊圖像出來。
#　size: 填充寬度和高度，可以為一個值(寬度和高度都用給予同樣的延伸)，或是分別對應寬度和高度設定。
#　padding: 參照影像Pad部分
#　pad_if_needed: 是否需要填充，True or False
#　fill: 參照影像Pad部分
#　padding_mode: 參照影像Pad部分
size=(250, 300)
transform = transforms.Compose([
    transforms.RandomCrop(size)])
new_img = transform(img_pil)
new_img

In [ ]:
# 6. 影像 RandomHorizontalFlip 和 RandomVerticalFlip
# 圖片(PIL Image)會在給定的機率下隨機進行水平或是垂直翻轉。
transform = transforms.Compose([
    transforms.Resize((350,350)),
    transforms.RandomHorizontalFlip(p=0.9),])
new_img = transform(img_pil)
new_img.show()
transform = transforms.Compose([
    transforms.Resize((300,400)),transforms.RandomVerticalFlip(p=0.9),])
new_img = transform(img_pil)
new_img

In [ ]:
# 7.  影像 RandomResizedCrop :圖片(PIL Image)會在給定的機率下隨機 裁切到隨機給定的大小並且resize到設定的大小。
# size: 圖片最後要輸出的大小。
# scale: 裁切圖片為原始突變的比例(default為0.08–1.0)
# ratio: 裁切圖片的原始長寬比(default:3/4–4/3)。
# interpolation: check 影像 Resize。
transform = transforms.Compose([transforms.RandomResizedCrop((300, 500))])
new_img = transform(img_pil)
new_img

In [ ]:
# 8.影像 TenCrop: 圖片(PIL Image)裁減一張圖得四個角圖片以及中間的圖片到指定大小(size)，並且進行水平或是垂直翻轉等。
# size: 圖片最後要輸出的大小。
# vertical_flip: 是否垂直翻轉，預設為水平翻轉(default:False)
UNIT_SIZE=200
size = (100, UNIT_SIZE)
transform = transforms.Compose([transforms.TenCrop(size, vertical_flip=False)])
new_img = transform(img_pil)
delta = 50 
new_img_2 = Image.new("RGB", (UNIT_SIZE*10+delta, 100))
top_right = 0
for im in new_img:
    new_img_2.paste(im, (top_right, 0)) 
    top_right += UNIT_SIZE + int(delta/10) 
new_img_2

In [ ]:
#  9 影像 GaussianBlur :圖片(PIL Image or torch tensor)高斯模糊化 如果為torch tensor影像必須為[…, C, H, W]。
# 圖片(PIL Image or torch tensor)高斯模糊化 如果為torch tensor影像必須為[…, C, H, W]。
# kernel_size: 高斯kernel的大小。
# sigma: 高斯kernel生成的標準差，sigma值需為 1. float: (float)，
# sigma固定在設定的float值 2. tuple: (min, max)，sigma在(min, max)隨機取出一個值。
transform = transforms.Compose([transforms.GaussianBlur(7,3) ])
new_img = transform(img_pil)
new_img

In [ ]:
# 10 影像 Pad :以圖片(PIL Image)外部往外延伸填充寬度和高度，填充值為pad值。
# padding: 填充寬度和高度，可以為一個值(四個邊都用給予同樣的延伸)，或是分別對應四個邊設定。
# fill: 填充的值 設定一個值則是所有channel都填這個值 或是分別對三個channel分別設定， 須設定padding_mode=constant
# padding_mode: 填充模式 . constant: 填充固定數字 . edge:邊緣的值直接往外延伸 . 
#               reflect: 從邊緣往內一個pixel進行鏡射 . symmetric:從邊緣鏡射
padding = (10, 5, 40, 20)
transform = transforms.Compose([transforms.Resize((100,150)),
    transforms.Pad(padding, fill=0,padding_mode="constant"), ])
new_img = transform(img_pil)
new_img.show()
padding = 40
transform = transforms.Compose([transforms.Resize((100,150)),
    transforms.Pad(padding, fill=100,padding_mode="constant"),])   # color image: fill=(100,200,255)
new_img = transform(img_pil)
new_img.show()
padding = (40, 40, 40, 40)
transform = transforms.Compose([transforms.Resize((100,150)),
    transforms.Pad(padding, padding_mode="edge"), ])
new_img = transform(img_pil)
new_img.show()
padding = (40, 40, 40, 40)
transform = transforms.Compose([transforms.Resize((100,150)),
    transforms.Pad(padding, padding_mode="symmetric"), ]) # symmetric :對稱
new_img = transform(img_pil); new_img

In [ ]:
# 10 影像 RandomCrop 以圖片(PIL Image)中隨機裁減一塊圖像出來。
# 參數設定:
# size: 填充寬度和高度，可以為一個值(寬度和高度都用給予同樣的延伸)，或是分別對應寬度和高度設定。
# padding: 參照影像Pad部分
# pad_if_needed: 是否需要填充，True or False
# fill: 參照影像Pad部分
# padding_mode: 參照影像Pad部分
size=(400, 200)
transform = transforms.Compose([transforms.RandomCrop(size)])
new_img = transform(img_pil); new_img

In [ ]:
# 11 影像 RandomAffine : 圖片(PIL Image or torch tensor)保持中心不變的圖像的隨機仿射變換。
# degrees: 旋轉角度， 設定為0代表不做圖片旋轉。 
#   1. float or int: 角度在(-degrees,+degrees)隨機取一個。 2. tuple: (min, max)，角度在(min, max)隨機取出一個值。
# translate: 水平和垂直平移，defalut為不做平移。 ex: translate=(a, b)，
#   水平部分會隨機在(-img_widtha, img_widtha) 平移 垂直部分會隨機在(-img_heightb, img_heightb) 平移
# scale: 縮放參數，為一個區段設定，defalut為keep原始圖片大小。 (a,b)縮放參數會在a-b之間隨機抽出一個數字。
# shear: 圖像裁減參數，可以參考Crop的設定，defalut為不做裁減。
# resample:An optional resampling filter 參考 https://pillow.readthedocs.io/en/latest/handbook/concepts.html#filters
# fillcolor:圖像外部填充的顏色。 (Tuple for RGB Image and int for grayscale) 。
transform = transforms.Compose([transforms.Resize((100,150)),
    transforms.RandomAffine(degrees=(-30,30), translate=(0, 0.5), 
                            scale=(0.4, 0.5), shear=(0,0), fillcolor=(0,255,255))])
new_img = transform(img_pil); new_img

In [ ]:
# 12 影像 Grayscale 和 RandomGrayscale  : Grayscale將圖片(PIL Image or torch tensor)轉換成灰階。
# Grayscale 參數設定:
# — num_output_channels (int，(1 or 3)): 
# 輸出圖像要幾個channel * 1: image is single channel * 3: image is 3 channel with r == g == b
# RandomGrayscale 參數設定:
# — p: 圖片要進行轉換灰階的機率。
# note: RandomGrayscale和 Grayscale不同，如果輸入是channel數是1，輸出的灰階則是1個chnnel，
# 如果輸入是三個channel，則輸出3 channel with r == g == b。
transform = transforms.Compose([transforms.Resize((100,150)),
    transforms.Grayscale(num_output_channels=1)])
new_img = transform(img_pil)
new_img_array = np.array(new_img)
print("original shape:", np.array(img_pil).shape)
print("shape:", new_img_array.shape); new_img.show()
transform = transforms.Compose([transforms.Resize((100,150)),
    transforms.Grayscale(num_output_channels=3)])
new_img = transform(img_pil)
new_img_array = np.array(new_img)
print("original shape:", np.array(img_pil).shape)
print("shape:", new_img_array.shape);new_img.show()
ransform = transforms.Compose([ transforms.Resize((100,150)),
    transforms.RandomGrayscale(p=0.9)])
new_img = transform(img_pil)
new_img_array = np.array(new_img)
print("original shape:", np.array(img_pil).shape)
print("shape:", new_img_array.shape);new_img

In [ ]:
# 13 影像 RandomPerspective 圖片(PIL Image or torch tensor)在給定的機率執行給定圖像的隨機透視變換。
# 參數設定: # — distortion_scale (float): 控制失真(distortion)程度，範圍為0–1，default:0.5。
#— p (float):執行轉換的機率，default: 0.5。 — interpolation (int):
# Interpolation type。 — fill (n-tuple or int or float): 當圖扭曲後，圖外滿要填滿的值。
transform = transforms.Compose([transforms.Resize((200,300)),
    transforms.RandomPerspective(distortion_scale=0.5, p=0.5, interpolation=2)])
new_img = transform(img_pil);new_img

In [ ]:
# 14 * 影像 ColorJitter  隨機調整圖片的亮度(brightness)、對比(contrast)、飽和度(saturation)和色調(hue)。
# brightness:(float or tuple(min, max))亮度從[max(0,1-brightness),1+brightness]隨機取一個值，或是[min, max]，須為非負數。
# contrast: (float or tuple(min, max))對比從[max(0, 1-contrast), 1+contrast]隨機取一個值，或是[min, max]，須為非負數。
# saturation: (float or tuple(min, max)) 飽和度 從[max(0, 1-saturation), 1+saturation]隨機取一個值，
# 或是[min, max]，須為非負數。
# hue: (float or tuple(min, max)) 色調 從[-hue, hue]隨機取一個值，或是[min, max],
# 但hue必須設定在[0,0.5]或-0.5<=min<=max<=0.5。
transform = transforms.Compose([transforms.Resize((100,150)),
    transforms.ColorJitter(brightness=(0, 5), contrast=(0, 5), saturation=(0, 5), hue=(-0.1, 0.1))])
new_img = transform(img_pil); new_img

In [ ]:
# 15 影像 RandomRotation  將圖片隨機旋轉。
# 參數設定:# degrees: (float or tuple(min, max)) 旋轉角度 從[-degrees, degrees]隨機取一個值，或是[min, max]。
# resample: 選轉後的圖，外圍補值方式。
# expand: (True or False) True: 將輸出圖況大到可以容納整張原始圖。 False: 輸出圖跟輸入圖一樣大，所以可能因為旋轉導致圖像被切斷。
# center: (n-tuple or int or float) 在圖片的哪個位置做為中心進行旋轉，default: None (圖的正中心旋轉)。
# fill: 同pad函數填滿方式，default: 0。
transform = transforms.Compose([
    transforms.RandomRotation(30, resample=Image.Resampling.BICUBIC, expand=False, center=(55, 5))])
new_img = transform(img_pil); new_img

In [ ]:
#  16. 影像 RandomApply :從給定的機率下隨機執行全部的設定轉換組合。
# 參數設定: # transforms (list or tuple or torch.nn.Module): list of transformations。
# p (float):probability，default: 0.5。
transform_set = [transforms.CenterCrop(200), transforms.Pad(100, padding_mode='symmetric'),
             transforms.RandomRotation(30), transforms.ColorJitter()]
transform = transforms.Compose([transforms.Resize((100,150)),
    transforms.RandomApply(transform_set, p=0.5)]); new_img = transform(img_pil); new_img

In [ ]:
# 17. 影像 RandomChoice 和 RandomOrder
# RandomChoice: 從設定的轉換組合隨機選取一個轉換執行。# RandomOrder: 從設定的轉換組合隨機打亂，全部執行。
transform_set = [ transforms.CenterCrop(200),transforms.Pad(20, padding_mode='symmetric'),
             transforms.RandomRotation(30),transforms.ColorJitter()]
transform = transforms.Compose([transforms.Resize((100,150)),
    transforms.RandomChoice(transform_set)]) ;  new_img = transform(img_pil); new_img.show()
transform_set = [ transforms.CenterCrop(200), transforms.Pad(20, padding_mode='symmetric'),
             transforms.RandomRotation(30), transforms.ColorJitter()]
transform = transforms.Compose([transforms.Resize((100,150)),transforms.RandomOrder(transform_set)])
new_img = transform(img_pil) ; new_img

In [ ]:
# ### CNN for real defect Image ============================
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchvision import transforms
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
class CustomImageDataset(Dataset): # 繼承Dataset 類別 
    def read_data_set(self):
        all_img_files = []
        all_labels = []
        class_names = []
        print('self.data_set_path=',self.data_set_path)
        print('os.listdir(self.data_set_path=',os.listdir(self.data_set_path)) # ['Cr', 'Pa', 'Sc']
        print('================================================================')
        for index, i in enumerate(os.listdir(self.data_set_path)):
            print('index = ',index)
            print('i=',  i) # i 依序 = ['Cr', 'Pa', 'Sc'] 
            class_names.append(i)
            for j in os.listdir(self.data_set_path+'/'+i):
                all_img_files.append(self.data_set_path+'/'+i+'/'+j)
                all_labels.append(index) # index :  Cr =0, Pa =1 , Sc =2
        print(all_img_files)        
        return all_img_files, all_labels, len(all_img_files), len(class_names)
      
    def __init__(self, data_set_path, transforms=None):
        self.data_set_path = data_set_path
        self.image_files_path, self.labels, self.length, self.num_classes = self.read_data_set()
        self.transforms = transforms
    def __getitem__(self, index):
        image = Image.open(self.image_files_path[index])
        image = image.convert("RGB")
        if self.transforms is not None:
            image = self.transforms(image)
        label = self.labels[index]
        return image, label
    def __len__(self):
        return self.length
class CustomConvNet(nn.Module):
    def __init__(self, num_classes):
        super(CustomConvNet, self).__init__()
        self.layer1 = self.conv_module(3, 16)
        self.layer2 = self.conv_module(16, 32)
        self.layer3 = self.conv_module(32, 64)
        self.layer4 = self.conv_module(64, 128)
        self.layer5 = self.conv_module(128, 256)
        self.gap = self.global_avg_pool(256, num_classes)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.gap(out)
        out = out.view(-1, num_classes)   # num_classes =3 分3類
        return out
    def conv_module(self, in_num, out_num):
        return nn.Sequential(
            nn.Conv2d(in_num, out_num, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_num),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
    def global_avg_pool(self, in_num, out_num):
        return nn.Sequential(
            nn.Conv2d(in_num, out_num, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_num),
            nn.LeakyReLU(),
            nn.AdaptiveAvgPool2d((1, 1)))
hyper_param_epoch = 10
hyper_param_learning_rate = 0.001
hyper_param_batch = 32
transforms_train = transforms.Compose([transforms.Resize((128, 128)),
                                       transforms.RandomRotation(10.),
                                       transforms.ToTensor()])
transforms_test = transforms.Compose([transforms.Resize((128, 128)),
                                      transforms.ToTensor()])
train_data_set = CustomImageDataset(data_set_path="./data_manufacture/train", transforms=transforms_train)
train_loader = DataLoader(train_data_set, batch_size=hyper_param_batch, shuffle=True)
test_data_set = CustomImageDataset(data_set_path="./data_manufacture/validation", transforms=transforms_test)
test_loader = DataLoader(test_data_set, batch_size=hyper_param_batch, shuffle=True)
num_classes = train_data_set.num_classes
custom_model = CustomConvNet(num_classes=num_classes).to(device)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(custom_model.parameters(), lr=hyper_param_learning_rate)
# trainong ==============================================
for e in range(hyper_param_epoch):
    for i_batch, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        # Forward pass
        outputs = custom_model(images)
        loss = criterion(outputs, labels)
        #print(loss)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Epoch [{}/{}], Loss: {:.4f}'.format(e + 1, hyper_param_epoch, loss.item()))
# Test the model
custom_model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels  in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = custom_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += len(labels)
         
    print('Test Accuracy of the model on the {} test images: {} %'.format(total, 100 * correct / total))